In [ ]:
import os
import json
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from jinja2 import Template

# Lade PDF
pdf_path = "moduldatenbank.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Splitte in kleinere Chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
docs = splitter.split_documents(documents)

# Embeddings mit gemma:2b
embeddings = OllamaEmbeddings(model="gemma:2b")

# Erstelle Chroma-Datenbank
db = Chroma.from_documents(docs, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# LLM-Setup
llm = OllamaLLM(
    base_url="http://134.96.217.20:53100",
    model="gemma:2b",
    temperature=0.5,
    top_p=0.8,
    top_k=10,
    repeat_penalty=1.1,
    presence_penalty=1.2
)

# Prompt-Template
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=(
        "Hier ist ein Auszug aus einem Dokument:\n"
        "{context}\n\n"
        "Beantworte bitte diese Frage basierend auf diesem Kontext, in einfacher, kindgerechter Sprache:\n"
        "{question}\n\n"
        "Antwort:"
    )
)

# Retrieval-Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

print("🤖 Chatbot gestartet! Tipp 'exit' oder 'quit' zum Beenden.\n")

# Chat-Schleife
while True:
    user_input = input("Du: ")
    if user_input.lower() in ["exit", "quit"]:
        print("🤖 Bot: Auf Wiedersehen!")
        break

    try:
        response = qa_chain.run(user_input)
        print("🤖 Bot:", response)

    except Exception as e:
        print(f"⚠️ Fehler: {e}")
